In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def cost_function(params):
    print("Evaluating parameters:", params)
    learning_rate = params[0]
    dense_units = int(params[1])  # Cast to integer for Dense layer
    # batch_size = int(params[1])   # Cast to integer for batch size

    # Data generators
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_data = train_datagen.flow_from_directory(
        '/kaggle/input/data-mal/data/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary',
        shuffle=True
    )

    val_data = val_datagen.flow_from_directory(
        '/kaggle/input/data-mal/data/val',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary'
    )

    # Model definition
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(dense_units, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        train_data,
        epochs=50,
        validation_data=val_data,
        verbose=0
    )

    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy  # Return positive accuracy for maximization

In [3]:
# Initial population
meow = np.array([
    [0.001, 128],
    [0.01, 64],
    [0.0001, 256],
    [0.005, 128],
    [0.02, 96]
])

In [4]:
def jaya_optimization():
    dim = 2  # Number of parameters
    lb = [0.0001, 32]  # Lower bounds [learning rate, dense units]
    ub = [0.1, 512]    # Upper bounds
    population = meow.copy().astype(float)
    
    # Evaluate initial population
    f = np.array([cost_function(ind) for ind in population])
    
    print("\nInitial Fitness Values (f):")
    print(f)

    for gen in range(4):
        best_idx = np.argmax(f)
        worst_idx = np.argmin(f)
        best = population[best_idx]
        worst = population[worst_idx]

        new_population = np.zeros_like(population)
        for i in range(len(population)):
            # Generate r1 and r2 ONCE per individual (shared across parameters)
            r1, r2 = np.random.rand(), np.random.rand()
            # r1 = np.random.rand()
            print(f"\nGeneration {gen+1}, Individual {i+1}: r1={r1:.4f}, r2={r2:.4f}")
            # print(f"\nGeneration {gen+1}, Individual {i+1}: r1={r1:.4f}, r2={1-r1:.4f}")

            for j in range(dim):
                # Use the same r1 and r2 for both parameters
                new_val = population[i, j] + r1*(best[j] - abs(population[i, j])) - r2*(worst[j] - abs(population[i, j]))
                new_population[i, j] = np.clip(new_val, lb[j], ub[j])
            
            # Ensure dense units are integers
            new_population[i, 1] = int(round(new_population[i, 1]))

        # Print new population after clipping
        print(f"\nGeneration {gen+1} - New Population (after clipping):")
        print(new_population)

        # Evaluate new solutions
        new_f = np.array([cost_function(ind) for ind in new_population])

        # Print new fitness values
        print(f"\nGeneration {gen+1} - New Fitness Values (new_f):")
        print(new_f)

        # Update population if new solution is better
        for i in range(len(population)):
            if new_f[i] > f[i]:
                population[i] = new_population[i]
                f[i] = new_f[i]

        best_acc = f[np.argmax(f)]
        print(f"\nGeneration {gen+1}, Best Accuracy: {best_acc:.4f}")

    best_idx = np.argmax(f)
    return population[best_idx], f[best_idx]

In [5]:
# Execute optimization
best_params, best_accuracy = jaya_optimization()
print(f"Optimal Parameters: Learning Rate = {best_params[0]:.6f}, Dense Units = {int(best_params[1])}")
print(f"Highest Validation Accuracy: {best_accuracy:.4f}")

Evaluating parameters: [1.00e-03 1.28e+02]
Found 2288 images belonging to 2 classes.
Found 762 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Evaluating parameters: [1.0e-02 6.4e+01]
Found 2288 images belonging to 2 classes.
Found 762 images belonging to 2 classes.
Evaluating parameters: [1.00e-04 2.56e+02]
Found 2288 images belonging to 2 classes.
Found 762 images belonging to 2 classes.
Evaluating parameters: [5.00e-03 1.28e+02]
Found 2288 images belonging to 2 classes.
Found 762 images belonging to 2 classes.
Evaluating parameters: [2.0e-02 9.6e+01]
Found 2288 images belonging to 2 classes.
Found 762 images belonging to 2 classes.

Initial Fitness Values (f):
[0.78608924 0.78608924 0.95013124 0.93963253 0.78608924]

Generation 1, Individual 1: r1=0.1151, r2=0.3221

Generation 1, Individual 2: r1=0.2981, r2=0.8819

Generation 1, Individual 3: r1=0.2509, r2=0.5951

Generation 1, Individual 4: r1=0.6366, r2=0.8812

Generation 1, Individual 5: r1=0.9961, r2=0.3686

Generation 1 - New Population (after clipping):
[[8.96454479e-04 1.43000000e+02]
 [1.49854569e-02 6.50000000e+01]
 [1.00000000e-04 3.32000000e+02]
 [5.40562567e-03